In [0]:
import pandas as pd
import numpy as np 

In [0]:
input_fname = "house1.csv"

In [0]:
df = pd.read_csv(input_fname, encoding="big5")
df.head()

,次,樓層,棟,號,出售面積(坪),持分土地面積(坪),標售單價(元/m2),標售總價(元),投標數,得標總價(元)/標售總價(元)
0,2019,3,C,_1,29.79,5.31,"69,118","6,806,071",2,1.0171
1,2019,5,C,_1,29.79,5.31,"69,118","6,806,071",3,1.0088
2,2019,6,C,_1,29.79,5.31,"69,118","6,806,071",4,1.0298
3,2019,7,C,_1,29.79,5.31,"69,723","6,865,651",6,1.1055
4,2019,4,D,_1,29.79,5.31,"68,543","6,749,470",5,1.0214


In [0]:
to_int = lambda x: int(x.replace(",",""))

In [0]:
df["標售單價(元/m2)"] = df["標售單價(元/m2)"].apply(to_int)
df["標售總價(元)"] = df["標售總價(元)"].apply(to_int)

In [0]:
df_op = df.copy()

In [0]:
df.head()

,次,樓層,棟,號,出售面積(坪),持分土地面積(坪),標售單價(元/m2),標售總價(元),投標數,得標總價(元)/標售總價(元)
0,2019,3,C,_1,29.79,5.31,69118,6806071,2,1.0171
1,2019,5,C,_1,29.79,5.31,69118,6806071,3,1.0088
2,2019,6,C,_1,29.79,5.31,69118,6806071,4,1.0298
3,2019,7,C,_1,29.79,5.31,69723,6865651,6,1.1055
4,2019,4,D,_1,29.79,5.31,68543,6749470,5,1.0214


In [0]:
no_2019 = np.array(df["次"]==2019)
no_2020 = np.array(df["次"]==2020)

In [0]:
y_inter = np.array(df.loc[no_2019,"投標數"].astype(np.float32))

In [0]:
y = np.array(df.loc[no_2019,"得標總價(元)/標售總價(元)"].astype(np.float32))

In [0]:
feature_names_list = []
X_list = []

# 離散變數 ["棟", "號"]

In [0]:
from sklearn.preprocessing import OneHotEncoder

In [0]:
cnames = ["棟", "號"]

In [0]:
ohe = OneHotEncoder(sparse=False)
X = ohe.fit_transform(df[cnames])

In [0]:
ohe.get_feature_names()

array(['x0_A', 'x0_B', 'x0_C', 'x0_D', 'x1__1', 'x1__2', 'x1__3', 'x1__5'],
      dtype=object)

In [0]:
feature_names = []
for i, cn in enumerate(ohe.get_feature_names()):
  sp = cn.split("_")
  k = cnames[int(sp[0].replace("x",""))]+"_"+("".join(sp[1:]))
  feature_names.append(k)
feature_names

['棟_A', '棟_B', '棟_C', '棟_D', '號_1', '號_2', '號_3', '號_5']

In [0]:
feature_names_list += feature_names
X_list.append(X) 

# 連續變數

In [0]:
cnames = [ '樓層', '出售面積(坪)', '持分土地面積(坪)', '標售單價(元/m2)', ]

In [0]:
from sklearn.preprocessing import StandardScaler

In [0]:
ss = StandardScaler()

In [0]:
X = ss.fit_transform(df[cnames])

In [0]:
feature_names_list += cnames
X_list.append(X) 

# 總和

In [0]:
X = np.concatenate(X_list, 1)

In [0]:
feature_names = feature_names_list

In [0]:
X.shape, len(feature_names)

((42, 12), 12)

# 預測投標數

In [0]:
X_train, y_train = X[no_2019], y_inter
X_test = X[no_2020]

In [0]:
from sklearn.ensemble import RandomForestRegressor
submodel = RandomForestRegressor()
submodel.fit(X_train,y_train)
submodel.score(X_train,y_train)
y_test_pred = submodel.predict(X_test)

In [0]:
n_bit_pred = np.round(y_test_pred)
n_bit_pred

array([ 7., 12.,  7., 10.,  4.,  5.,  4.,  5.,  4.,  5.,  3.,  8., 14.,
        8., 12.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  9.])

In [0]:
np.mean(n_bit_pred)

6.84

In [0]:
n_bit = np.concatenate((y_inter,n_bit_pred)).reshape((-1,1))

In [0]:
n_bit_norm = StandardScaler().fit_transform(n_bit)
n_bit_norm.shape

(42, 1)

In [0]:
X = np.concatenate((X,n_bit_norm), 1)
X.shape

(42, 13)

# Model

In [0]:
X_train, y_train = X[no_2019], y
X_test = X[no_2020]

In [0]:
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

In [0]:
def get_y_test_pred(model, print_name=""):
  model = model()
  model.fit(X_train,y_train)
  print("R-sq:", model.score(X_train,y_train) )
  y_train_pred = model.predict(X_train)
  err_mean = np.mean((y_train_pred - y_train)**2)
  err_std = np.std((y_train_pred - y_train)**2)
  print("[{}] 誤差:0% ~ {:.2f}%".format( print_name, (err_mean+err_std*3)*100 ))
  y_test_pred =model.predict(X_test)
  return y_test_pred

In [0]:
models = [SVR, RandomForestRegressor, ExtraTreesRegressor]
print_names = ["SVR", "RandomForestRegressor", "ExtraTreesRegressor"]
y_test_pred_list = []
for model,name in zip(models,print_names):
  y_test_pred_list.append( get_y_test_pred(model, print_name=name) )
  print("*"*64)


R-sq: 0.39016878283553325
[SVR] 誤差:0% ~ 1.76%
****************************************************************
R-sq: 0.8635614138547315
[RandomForestRegressor] 誤差:0% ~ 0.77%
****************************************************************
R-sq: 1.0
[ExtraTreesRegressor] 誤差:0% ~ 0.00%
****************************************************************


In [0]:
y_test_pred_list[1]

array([1.095926  , 1.20884903, 1.096175  , 1.19888703, 1.05825001,
       1.17623099, 1.05825001, 1.17623099, 1.05825001, 1.17623099,
       1.05956202, 1.12394302, 1.23105703, 1.12444102, 1.19723302,
       1.084936  , 1.17271301, 1.084936  , 1.17271301, 1.084936  ,
       1.17271301, 1.084936  , 1.17271301, 1.10998701, 1.12600601])

# 匯出

In [0]:
del df_op["投標數"]
del df_op["得標總價(元)/標售總價(元)"]
df_op = df_op.iloc[no_2020,:]

In [0]:
for i,model_name in enumerate(print_names):
  df_op[model_name+"_"+"投標數(假設為)"] = n_bit_pred
  df_op[model_name+"_""得標總價/標售總價(預測)"] = y_test_pred_list[i]
  df_op[model_name+"_""得標總價(預測)"] = df_op["標售總價(元)"]*y_test_pred_list[i]

In [0]:
df_op.head()

,次,樓層,棟,號,出售面積(坪),持分土地面積(坪),標售單價(元/m2),標售總價(元),SVR_投標數(假設為),SVR_得標總價/標售總價(預測),SVR_得標總價(預測),RandomForestRegressor_投標數(假設為),RandomForestRegressor_得標總價/標售總價(預測),RandomForestRegressor_得標總價(預測),ExtraTreesRegressor_投標數(假設為),ExtraTreesRegressor_得標總價/標售總價(預測),ExtraTreesRegressor_得標總價(預測)
17,2020,8,C,_1,29.79,5.31,69723,6865651,7.0,1.127867,7.743539e+06,7.0,1.095926,7.524245e+06,7.0,1.097382,7.534242e+06
18,2020,8,C,_2,32.10,5.77,67803,7195300,12.0,1.162235,8.362626e+06,12.0,1.208849,8.698031e+06,12.0,1.211268,8.715437e+06
19,2020,9,C,_1,29.79,5.31,70873,6978853,7.0,1.134301,7.916119e+06,7.0,1.096175,7.650044e+06,7.0,1.097382,7.658468e+06
20,2020,9,C,_2,32.10,5.77,68983,7320490,10.0,1.154060,8.448288e+06,10.0,1.198887,8.776440e+06,10.0,1.182559,8.656912e+06
21,2020,1,C,_1,29.79,5.31,72053,7095034,4.0,1.132945,8.038282e+06,4.0,1.058250,7.508320e+06,4.0,1.033809,7.334910e+06


In [0]:
df_op.to_csv("house_pred_1.csv", encoding="big5")

# 分析

In [0]:
from sklearn.svm import SVR
lr_ = SVR(kernel="linear")
lr_.fit(X_train,y_train)
coef = lr_.coef_[0]

In [0]:
coef

array([ 0.01905557, -0.01514689,  0.00107597, -0.00498465,  0.0054159 ,
        0.00973099, -0.01514689,  0.        , -0.0131775 ,  0.01112483,
        0.01144824,  0.01613987,  0.03121172])

In [0]:
feature_names += ["投標數(假設為)"]

In [0]:
feature_names = np.array(feature_names)

In [0]:
coef_sort = np.sort(coef)

In [0]:
rank = list(feature_names[ np.argsort(coef)])
rank

['棟_B',
 '號_3',
 '樓層',
 '棟_D',
 '號_5',
 '棟_C',
 '號_1',
 '號_2',
 '出售面積(坪)',
 '持分土地面積(坪)',
 '標售單價(元/m2)',
 '棟_A',
 '投標數(假設為)']

In [0]:
dict(zip(range(len(rank),0,-1), [(r,round(c,4)) for r,c in zip(rank, coef_sort)]  ))

{1: ('投標數(假設為)', 0.0312),
 2: ('棟_A', 0.0191),
 3: ('標售單價(元/m2)', 0.0161),
 4: ('持分土地面積(坪)', 0.0114),
 5: ('出售面積(坪)', 0.0111),
 6: ('號_2', 0.0097),
 7: ('號_1', 0.0054),
 8: ('棟_C', 0.0011),
 9: ('號_5', 0.0),
 10: ('棟_D', -0.005),
 11: ('樓層', -0.0132),
 12: ('號_3', -0.0151),
 13: ('棟_B', -0.0151)}